# Generic Box
This code communicates with a component. The component was launched with the command below from the root of the repository:

```shell
$ docker run --rm -it -p 4061:8061 -v ./src/external1.py=/workspace/external.py generic
```


In [38]:
import matplotlib.pyplot as plt
import PIL.Image as PIL_image
import numpy as np
import cv2
from scipy.io import loadmat,savemat

from importlib.machinery import SourceFileLoader
#In case of importing files from other directory
cotrakcer_pb2 = SourceFileLoader("cotracker_pb2","../protos/cotracker_pb2.py").load_module()
cotrakcer_pb2_grpc = SourceFileLoader("cotracker_pb2_grpc","../protos/cotracker_pb2_grpc.py").load_module()

import argparse
import grpc
import io


## Set parameters (GRPC) and data
Set IP and port of the service (the same used in docker run), input data (image in a matfile) and call the service

In [39]:
from PIL import Image

#target='printart.isr.ist.utl.pt:8061'
target='localhost:8061'

# List of file paths (example paths — replace with your actual ones)
video_path = "./cut.mp4"

with open(video_path, "rb") as f:
    video_bytes = f.read()

request = cotrakcer_pb2.CoTrackerRequest(
    video=video_bytes,
    grid_size=32)

channel_opt = [('grpc.max_send_message_length', 512 * 1024 * 1024), ('grpc.max_receive_message_length', 512 * 1024 * 1024)]
channel=grpc.insecure_channel(target,options=channel_opt)
estimator_stub = cotrakcer_pb2_grpc.CoTrackerServiceStub(channel)

FileNotFoundError: [Errno 2] No such file or directory: './cut.mp4'

#### Do your task. Example: read an image from a matlab file and send it to process

In [34]:
#Name of the method in the service
response = estimator_stub.Forward(request)
#After finishing channel.close

In [ ]:
import torch
torch.load(io.BytesIO(response.tracks)).shape
torch.load(io.BytesIO(response.visibility))

tensor([[[ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True],
         ...,
         [ True,  True,  True,  ...,  True, False, False],
         [False,  True,  True,  ...,  True, False, False],
         [False,  True,  True,  ...,  True, False, False]]])

I0000 00:00:1757355439.825990   87464 chttp2_transport.cc:1335] ipv6:%5B::1%5D:8061: Got goaway [2] err=UNAVAILABLE:GOAWAY received; Error code: 2; Debug Text: Cancelling all calls {grpc_status:14, http2_error:2}
